In [ ]:
import sample_generators
from tqdm.notebook import tqdm
import numpy as np
import multiprocessing as mp


tree = [0]+list(range(9))
print(tree)
equivalence_class = sample_generators.generate_equivalence_class(sample_generators.makeEdges(tree))
w = 0.5
noisy_nodes = [0,2,4,6,8]
noise_variance = 2
SigmaQ, rho_min, RHO_MAX, Q = sample_generators.getStatistics(tree, w, noisy_nodes, noise_variance)


m_error_p = 1
m_error_ps = []
k_error_ps = []
sample_range = []
pbar = tqdm()
tqdm.refresh

num_samples = 1100
STEP = 750
MAX_ITER = 10**3

def log_result(result):
    global pbar
    global errors
    pbar.update()
    errors = errors + result

def do_async(args, MAX_ITER):
    num_processors = 4
    p=mp.Pool(processes = num_processors)
    for i in range(MAX_ITER):
        p.apply_async(sample_generators.parallelGaussianTest, args = args, callback = log_result) 
    p.close()
    p.join()

while(m_error_p > 10**-3):
    errors = np.zeros(2)
    args = (num_samples, rho_min, RHO_MAX, SigmaQ, equivalence_class, Q)
    
    pbar.reset(total = len(range(MAX_ITER)))
    if __name__ ==  '__main__': 
        do_async(args, MAX_ITER)
    pbar.refresh()

    errors = errors/MAX_ITER
    m_error_p = errors[0]
    k_error_p = errors[1]
    m_error_ps.append(m_error_p)
    k_error_ps.append(k_error_p)
    sample_range.append(num_samples)

    print('samples:', num_samples, 'k errors:', k_error_p, 'm errors:', m_error_p)

    num_samples += STEP
